# SpiNNaker2

SpiNNaker2 is a neuromorphic chip based around a grid of ARM Cortex-M4F processors which are tighly coupled with accelerators and a network-on-chip optimized for, but not limited to transmission of spikes.

Running this requires the following library and installed SpiNNaker2 hardware: [py-spinnaker2](https://gitlab.com/spinnaker2/py-spinnaker2). Currently the branch `py-spinnaker2_nir` is required.

## Create a graph

In [ ]:
import numpy as np
from spinnaker2 import s2_nir

import nir

nir_model = nir.NIRGraph(
    nodes={
        "in": nir.Input(input_type=np.array([3])),
        "affine": nir.Affine(
            weight=np.array([[8, 2, 10], [14, 3, 14]]).T * 32,
            bias=np.array([0, 8]) * 32,
        ),
        "lif": nir.LIF(
            tau=np.array([4] * 2),
            r=np.array([1.25, 0.8]),
            v_leak=np.array([0.5] * 2),
            v_threshold=np.array([5] * 2) * 32,
        ),
        "out": nir.Output(output_type=np.array([2])),
    },
    edges=[("in", "affine"), ("affine", "lif"), ("lif", "out")],
)
nir.write("nir_model.hdf5", nir_model)

print(nir_model)
print("read back")
net = s2_nir.from_nir(nir_model)
print(net)


## Import to Spinnaker2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spinnaker2 import hardware, helpers, s2_nir

import nir

np.random.seed(123)

timesteps = 50

# load NIR model
nir_model = nir.read("nir_model.hdf5")
print(nir_model)
net, inp, outp = s2_nir.from_nir(
    nir_model, outputs=["v", "spikes"], discretization_timestep=1, conn_delay=0
)


# Create some input spikes
input_size = inp[0].size
input_spikes = {}
input_data = np.random.randn(input_size, timesteps)
input_data = (input_data > 1) * 1
print(input_data)

for i in range(input_size):
    input_spikes[i] = input_data[i].nonzero()[0].tolist()

inp[0].params = input_spikes


# Load up hardware + run
hw = hardware.SpiNNaker2Chip(eth_ip="192.168.1.52")  # use ethernet

timesteps += 3
hw.run(net, timesteps)


# get results and plot
spike_times = outp[0].get_spikes()
voltages = outp[0].get_voltages()

fig, axs = plt.subplots(1, 3, sharex=True)

indices, times = helpers.spike_times_dict_to_arrays(input_spikes)
axs[0].plot(times, indices, ".")
axs[0].set_xlim(0, timesteps)
axs[0].set_ylim(-0.5, len(input_spikes.keys()) - 0.5)
axs[0].set_ylabel("neuron")

indices, times = helpers.spike_times_dict_to_arrays(spike_times)
axs[1].plot(times, indices, ".")
axs[1].set_xlim(0, timesteps)
axs[1].set_ylim(-0.5, outp[0].size - 0.5)
axs[1].set_ylabel("neuron")

for i, _ in voltages.items():
    axs[2].plot(_, label=i)
    axs[2].set_xlabel("time step")
    axs[2].set_ylabel("membrane potential")
    axs[2].set_xlim(0, timesteps)
    axs[2].legend()
    axs[2].grid()

plt.show()
